In [29]:
from pyspark.sql import SparkSession

# Iniciar a sessão Spark
# Definir um tamanho máximo de memória disponível para uso pelo driver
spark = SparkSession.builder \
    .appName("Bolsa Familia Data Analysis") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Atribui o SparkContext à variável 'sc'
sc = spark.sparkContext

# Definir o caminho do arquivo CSV
csv_file_path = 'BolsaFamilia/202306_NovoBolsaFamilia.csv'

# Ler os dados do CSV para um DataFrame. Aqui, assumimos que o arquivo CSV possui um cabeçalho.
# Também inferimos automaticamente o esquema dos dados. Adicionalmente, como o delimitador
# padrão de um arquivo CSV é uma vírgula e o arquivo que estamos usando pode ter um delimitador diferente
# (como ponto e vírgula), especificamos isso na opção 'sep'. Também indicamos o caractere de aspas
# com a opção 'quote'. Para lidar com questões de codificação de caracteres que podem surgir ao trabalhar
# com arquivos em formatos diferentes de UTF-8, usamos a opção 'encoding' para definir a codificação
# correta, que neste caso é 'ISO-8859-1'.
bolsa_familia_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("encoding", "ISO-8859-1") \
    .option("sep", ";") \
    .option("quote", "\"") \
    .csv(csv_file_path)

# Mostrar as primeiras linhas do DataFrame para verificação. Aqui, usamos 'truncate=False' para garantir
# que o Spark exiba o conteúdo completo das células do DataFrame, o que é útil para visualizar dados
# com conteúdo mais extenso.
bolsa_familia_df.show(truncate=False)

+---------------+--------------+---+----------------------+--------------+--------------+--------------+-------------------------------+-------------+
|MÊS COMPETÊNCIA|MÊS REFERÊNCIA|UF |CÓDIGO MUNICÍPIO SIAFI|NOME MUNICÍPIO|CPF FAVORECIDO|NIS FAVORECIDO|NOME FAVORECIDO                |VALOR PARCELA|
+---------------+--------------+---+----------------------+--------------+--------------+--------------+-------------------------------+-------------+
|202306         |202306        |AC |643                   |ACRELANDIA    |***.833.642-**|16167611395   |ABIGAIL DAGMAR MACHADO         |850,00       |
|202306         |202306        |AC |643                   |ACRELANDIA    |null          |16122034321   |ABRAAO AMORA SALGUEIRO         |600,00       |
|202306         |202306        |AC |643                   |ACRELANDIA    |***.355.042-**|16121660806   |ABRAAO DA PIEDADE DO NASCIMENTO|750,00       |
|202306         |202306        |AC |643                   |ACRELANDIA    |***.050.782-**|16094

In [30]:
# Registrar o DataFrame como uma tabela temporária
bolsa_familia_df.createOrReplaceTempView("bolsa_familia")

In [7]:
# Consulta simples que seleciona algumas colunas da tabela
result_df = spark.sql("""
SELECT `NOME MUNICÍPIO`, `NOME FAVORECIDO`, `VALOR PARCELA`
FROM bolsa_familia
""")
result_df.show(truncate=False)

+--------------+-------------------------------+-------------+
|NOME MUNICÍPIO|NOME FAVORECIDO                |VALOR PARCELA|
+--------------+-------------------------------+-------------+
|ACRELANDIA    |ABIGAIL DAGMAR MACHADO         |850,00       |
|ACRELANDIA    |ABRAAO AMORA SALGUEIRO         |600,00       |
|ACRELANDIA    |ABRAAO DA PIEDADE DO NASCIMENTO|750,00       |
|ACRELANDIA    |ABRAO KEMPNER RUMANZKI         |600,00       |
|ACRELANDIA    |ACHILLA MARIA SOUZA ANDRADE    |700,00       |
|ACRELANDIA    |ADALCICLEIA BARROS DE ARAUJO   |650,00       |
|ACRELANDIA    |ADALGIZA NOGUEIRA DOS SANTOS   |600,00       |
|ACRELANDIA    |ADAO DE SOUZA PIEDADE          |600,00       |
|ACRELANDIA    |ADAO INES DE MELO              |600,00       |
|ACRELANDIA    |ADECILDO DA SILVA CORREIA      |650,00       |
|ACRELANDIA    |ADEIDE RODRIGUES DAMASCENO     |700,00       |
|ACRELANDIA    |ADELINA XUITES JACOB           |650,00       |
|ACRELANDIA    |ADEMIR DANTAS DOS SANTOS JUNIOR|600,00 

In [8]:
# Executar uma consulta SQL e mostrar os resultados diretamente
spark.sql("SELECT * FROM bolsa_familia").show()

+---------------+--------------+---+----------------------+--------------+--------------+--------------+--------------------+-------------+
|MÊS COMPETÊNCIA|MÊS REFERÊNCIA| UF|CÓDIGO MUNICÍPIO SIAFI|NOME MUNICÍPIO|CPF FAVORECIDO|NIS FAVORECIDO|     NOME FAVORECIDO|VALOR PARCELA|
+---------------+--------------+---+----------------------+--------------+--------------+--------------+--------------------+-------------+
|         202306|        202306| AC|                   643|    ACRELANDIA|***.833.642-**|   16167611395|ABIGAIL DAGMAR MA...|       850,00|
|         202306|        202306| AC|                   643|    ACRELANDIA|          null|   16122034321|ABRAAO AMORA SALG...|       600,00|
|         202306|        202306| AC|                   643|    ACRELANDIA|***.355.042-**|   16121660806|ABRAAO DA PIEDADE...|       750,00|
|         202306|        202306| AC|                   643|    ACRELANDIA|***.050.782-**|   16094443293|ABRAO KEMPNER RUM...|       600,00|
|         202306|   

In [9]:
# Soma das parcelas por município
sum_df = spark.sql("""
SELECT `NOME MUNICÍPIO`, SUM(CAST(REPLACE(`VALOR PARCELA`, ',', '.') AS FLOAT)) AS total_valor
FROM bolsa_familia
GROUP BY `NOME MUNICÍPIO`
""")
sum_df.show(truncate=False)

+------------------------+-----------+
|NOME MUNICÍPIO          |total_valor|
+------------------------+-----------+
|CRUZEIRO DO SUL         |1.061035E7 |
|ESTRELA DE ALAGOAS      |2151199.0  |
|BATALHA                 |6848992.0  |
|CAMPO ALEGRE            |4174968.0  |
|DOIS RIACHOS            |1807040.0  |
|LAGOA DA CANOA          |2846837.0  |
|GUAJARA                 |2499744.0  |
|MONTEIROPOLIS           |1230435.0  |
|MAUES                   |1.1029124E7|
|IGREJA NOVA             |2637726.0  |
|ASSIS BRASIL            |1407480.0  |
|SAO MIGUEL DOS CAMPOS   |6794312.0  |
|PORTO WALTER            |1576825.0  |
|SAO GABRIEL DA CACHOEIRA|6399189.0  |
|RIO BRANCO              |2.9420177E7|
|FEIRA GRANDE            |3274435.0  |
|JUTAI                   |4016887.0  |
|OLHO D'AGUA DAS FLORES  |2974808.0  |
|MATRIZ DE CAMARAGIBE    |3187671.0  |
|INHAPI                  |2698601.0  |
+------------------------+-----------+
only showing top 20 rows



In [ ]:
# Soma das parcelas por município com formatação de moeda
sum_df = spark.sql("""
SELECT `NOME MUNICÍPIO`, 
       format_number(SUM(CAST(REPLACE(`VALOR PARCELA`, ',', '.') AS FLOAT)), 2) AS total_valor
FROM bolsa_familia
GROUP BY `NOME MUNICÍPIO`
""")
sum_df.show(truncate=False)

In [ ]:
# Média do valor das parcelas
avg_df = spark.sql("""
SELECT AVG(CAST(REPLACE(`VALOR PARCELA`, ',', '.') AS FLOAT)) AS avg_valor
FROM bolsa_familia
""")
avg_df.show(truncate=False)

In [ ]:
# Agregação por município para encontrar o total distribuído e o número de beneficiários
aggregated_df = spark.sql("""
SELECT `NOME MUNICÍPIO`, 
       COUNT(*) AS numero_beneficiarios,
       SUM(CAST(REPLACE(`VALOR PARCELA`, ',', '.') AS FLOAT)) AS total_distribuido
FROM bolsa_familia
GROUP BY `NOME MUNICÍPIO`
ORDER BY total_distribuido DESC
""")
aggregated_df.show(truncate=False)

In [6]:
# Selecionar todos os favorecidos que receberam mais de um valor específico
high_value_df = spark.sql("""
SELECT `NOME FAVORECIDO`, CAST(REPLACE(`VALOR PARCELA`, ',', '.') AS FLOAT) AS `VALOR_PARCELA`
FROM bolsa_familia
WHERE CAST(REPLACE(`VALOR PARCELA`, ',', '.') AS FLOAT) > 4000
""")
high_value_df.show(truncate=False)

+------------------------------------+-------------+
|NOME FAVORECIDO                     |VALOR_PARCELA|
+------------------------------------+-------------+
|ANTONIO MANOEL PEREIRA CONCEICAO    |4106.0       |
|CRISTIANE ALMEIDA LARANJEIRA        |4140.0       |
|GEAZI PEREIRA PIMENTA               |4390.0       |
|IRISLANDIA SOARES ARAUJO            |4198.0       |
|JOSE NALDAM SOARES BRITO            |4514.0       |
|MARINA SOARES FARIAS                |4006.0       |
|MATHEUS LUZ SILVA                   |4322.0       |
|PEDRO DOS SANTOS PINHEIRO DE ALMEIDA|4230.0       |
|SANDRO AUGUSTO PAVAO PEREIRA        |4040.0       |
|VIVIA MERCIA NUNES MOREIRA          |4130.0       |
|ROSENDO CAETANO DE SARGES           |4080.0       |
+------------------------------------+-------------+



## Exemplo do Uso de Cache

In [ ]:
# Carregar os dados e criar um DataFrame
df = spark.read.csv("path_to_data.csv", header=True, inferSchema=True)

# Registrar o DataFrame como uma tabela temporária
df.createOrReplaceTempView("my_table")

# Cache a tabela após a primeira operação de transformação
spark.sql("SELECT * FROM my_table WHERE some_column > some_value").cache().createOrReplaceTempView("filtered_table")

# Executar várias operações de transformação em sequência no DataFrame cacheado
# Como o DataFrame está cacheado, estas operações subsequentes serão mais rápidas

# Por exemplo, contar os registros
spark.sql("SELECT COUNT(*) FROM filtered_table").show()

# Calcular a média de uma coluna
spark.sql("SELECT AVG(numeric_column) FROM filtered_table").show()

# Agrupar por uma coluna e contar os valores
spark.sql("SELECT category_column, COUNT(*) FROM filtered_table GROUP BY category_column").show()

# Após as operações, é bom liberar o cache para evitar o uso desnecessário de memória
spark.catalog.uncacheTable("filtered_table")


## Interoperabilidade 

In [8]:
# Ler dados de um arquivo CSV
df_csv = spark.read.csv("arquivos/clientes.csv", header=True, inferSchema=True)
df_csv.createOrReplaceTempView("csv_table")

# Executar uma consulta SQL diretamente sobre os dados CSV
spark.sql("SELECT id_cliente, nome, sobrenome, email FROM csv_table").show()

+----------+-----------+----------+--------------------+
|id_cliente|       nome| sobrenome|               email|
+----------+-----------+----------+--------------------+
|         1|     Esther|  Teixeira|mariana858948@exa...|
|         2|    Mariane|   Barbosa|isissilva7264@exa...|
|         3|Ana Vitória|     Moura|fernandesjoaquim8...|
|         4|    Leandro|     Viana|brunosilva8705@ex...|
|         5|     Pietro|    Santos|ana-julia375761@e...|
|         6|    Clarice|    da Paz|duarteotavio7887@...|
|         7|Ana Vitória|  da Cunha|caue115510@exampl...|
|         8| Ana Sophia|   da Mota|moraesantonio7820...|
|         9|   Leonardo|    Fogaça|teixeiraenrico929...|
|        10|    Anthony|   Cardoso|eduardo553737@exa...|
|        11|    Natália|    Mendes|maite253689@examp...|
|        12|       Davi|      Dias|heloisa037394@exa...|
|        13|   Stephany|   Cardoso|jesusesther4336@e...|
|        14|      Laura|   da Cruz|maysaalves2332@ex...|
|        15| Ana Sophia|    da 

In [61]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

# Criar uma SparkSession
spark = SparkSession.builder.appName("JsonRead").getOrCreate()

# Ler o arquivo JSON original com a opção "multiline"
df_json = spark.read.option("multiline", "true").json("arquivos/clientes.json")

# Aplanar o array de clientes usando explode e acessar os objetos cliente
df_exploded = df_json.select(explode(df_json.clientes).alias("cliente"))

# Mostrar o esquema e os dados aplanados
df_exploded.printSchema()
# Assumindo que 'df_exploded' é o seu DataFrame que contém a coluna 'cliente' aplanada

# Criar uma visualização temporária
df_exploded.createOrReplaceTempView("clientes_view")

# Agora você pode executar consultas SQL usando a visualização temporária 'clientes_view'
# Por exemplo, para selecionar todos os clientes:
spark.sql("SELECT cliente.nome, cliente.endereco.rua, cliente.endereco.cidade FROM clientes_view").show(truncate=False)

root
 |-- cliente: struct (nullable = true)
 |    |-- celular: string (nullable = true)
 |    |-- cpf: string (nullable = true)
 |    |-- data_atualizacao: string (nullable = true)
 |    |-- data_criacao: string (nullable = true)
 |    |-- data_nascimento: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- numero: string (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |-- genero: string (nullable = true)
 |    |-- id_cliente: long (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- sobrenome: string (nullable = true)
 |    |-- status: boolean (nullable = true)
 |    |-- telefone: string (nullable = true)

+-----------+-------------+------+
|no

In [57]:
df_exploded.write.format("parquet").saveAsTable("clientes_table")

In [ ]:
# Demonstração
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SaveAsTableExample") \
    .config("spark.sql.warehouse.dir", "/caminho/para/o/diretorio/desejado") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
# Demonstração
df_exploded.write.format("parquet").save("/caminho/para/o/diretorio/desejado/clientes_table")

In [60]:
# Ler dados de um arquivo Parquet
df_parquet = spark.read.parquet("spark-warehouse/clientes_table/part-00000-8809baa0-abd3-48b2-a95a-7dab6fdc6078-c000.snappy.parquet")
df_parquet.createOrReplaceTempView("parquet_table")

# Executar uma consulta SQL diretamente sobre os dados Parquet
spark.sql("SELECT cliente.nome, cliente.sobrenome, cliente.endereco.rua FROM parquet_table").show()

+-----------+---------+-------------+
|       nome|sobrenome|          rua|
+-----------+---------+-------------+
|     Esther| Teixeira|Vila da Costa|
|    Mariane|  Barbosa|Vila da Costa|
|Ana Vitória|    Moura|Vila da Costa|
+-----------+---------+-------------+



In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.2.5.jar") \
    .getOrCreate()

In [5]:
# Conectar ao banco de dados PostgreSQL via JDBC
df_postgres = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://host.docker.internal:5432/01_pdv") \
    .option("dbtable", "public.clientes") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .load()

df_postgres.createOrReplaceTempView("clientes")

# Executar uma consulta SQL diretamente sobre os dados do PostgreSQL
spark.sql("SELECT * FROM clientes").show()

+----------+-----------+-----------+----------+---------------+--------------------+---------------+---------------+-----------+------+--------------------+--------------------+------+
|id_cliente|       nome|id_endereco| sobrenome|data_nascimento|               email|       telefone|        celular|        cpf|genero|        data_criacao|    data_atualizacao|status|
+----------+-----------+-----------+----------+---------------+--------------------+---------------+---------------+-----------+------+--------------------+--------------------+------+
|         1|     Esther|        163|  Teixeira|     2000-09-29|mariana858948@exa...|(65) 79034-9652|(22) 90597-2207|71284960331|     F|2023-10-22 01:59:...|2023-10-22 01:59:...|  true|
|         2|    Mariane|        152|   Barbosa|     2005-01-01|isissilva7264@exa...|(29) 25866-7664|(42) 62029-1129|38607941240|     M|2023-10-22 01:59:...|2023-10-22 01:59:...|  true|
|         3|Ana Vitória|         41|     Moura|     1972-05-02|fernandesjoa

## UDF's

In [67]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_date
from pyspark.sql.types import IntegerType
from datetime import datetime, date

# Inicializando uma SparkSession
spark = SparkSession.builder.appName("UDFExample").getOrCreate()

# Definindo uma função Python para calcular a idade
def calculate_age(birthdate):
    if isinstance(birthdate, datetime):
        birthdate = birthdate.date()
    elif isinstance(birthdate, str):
        birthdate = datetime.strptime(birthdate, "%Y-%m-%d").date()
    
    today = date.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

# Registrando a função como UDF
calculate_age_udf = udf(calculate_age, IntegerType())

# Lendo o arquivo CSV
df = spark.read.csv('arquivos/clientes.csv', header=True, inferSchema=True)

# Convertendo a coluna 'data_nascimento' para tipo data
df = df.withColumn("data_nascimento", to_date(col("data_nascimento"), "yyyy-MM-dd"))

# Adicionando a coluna 'idade' usando a UDF
df = df.withColumn("idade", calculate_age_udf(col("data_nascimento")))

# Selecionando colunas específicas
df.select("id_cliente", "nome", "idade").show()

# Salvando o DataFrame modificado
df.write.format("parquet").mode("ignore").save("arquivos/parquet/clientes_por_idade.parquet")

+----------+-----------+-----+
|id_cliente|       nome|idade|
+----------+-----------+-----+
|         1|     Esther|   23|
|         2|    Mariane|   18|
|         3|Ana Vitória|   51|
|         4|    Leandro|   66|
|         5|     Pietro|   75|
|         6|    Clarice|   32|
|         7|Ana Vitória|   25|
|         8| Ana Sophia|   58|
|         9|   Leonardo|   40|
|        10|    Anthony|   77|
|        11|    Natália|   78|
|        12|       Davi|   40|
|        13|   Stephany|   65|
|        14|      Laura|   66|
|        15| Ana Sophia|   49|
|        16|       Levi|   57|
|        17|      Alice|   41|
|        18|     Marina|   26|
|        19|     Danilo|   36|
|        20|     Marina|   73|
+----------+-----------+-----+
only showing top 20 rows



In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_date
from pyspark.sql.types import IntegerType
from datetime import datetime, date

# Inicializando uma SparkSession
spark = SparkSession.builder.appName("UDFExample").getOrCreate()

# Definindo uma função Python para calcular a idade
def calculate_age(birthdate):
    if isinstance(birthdate, datetime):
        birthdate = birthdate.date()
    elif isinstance(birthdate, str):
        birthdate = datetime.strptime(birthdate, "%Y-%m-%d").date()
    
    today = date.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

# Registrando a função como UDF no contexto SQL
spark.udf.register("calculate_age", calculate_age, IntegerType())

# Lendo o arquivo CSV
df = spark.read.csv('arquivos/clientes.csv', header=True, inferSchema=True)

# Convertendo a coluna 'data_nascimento' para tipo data
df = df.withColumn("data_nascimento", to_date(col("data_nascimento"), "yyyy-MM-dd"))

# Criando uma view temporária para usar SQL
df.createOrReplaceTempView("clientes")

# Usando a UDF em uma consulta SQL
spark.sql("SELECT id_cliente, nome, calculate_age(data_nascimento) AS idade FROM clientes").show()


+----------+-----------+-----+
|id_cliente|       nome|idade|
+----------+-----------+-----+
|         1|     Esther|   23|
|         2|    Mariane|   18|
|         3|Ana Vitória|   51|
|         4|    Leandro|   66|
|         5|     Pietro|   75|
|         6|    Clarice|   32|
|         7|Ana Vitória|   25|
|         8| Ana Sophia|   58|
|         9|   Leonardo|   40|
|        10|    Anthony|   77|
|        11|    Natália|   78|
|        12|       Davi|   40|
|        13|   Stephany|   65|
|        14|      Laura|   66|
|        15| Ana Sophia|   49|
|        16|       Levi|   57|
|        17|      Alice|   41|
|        18|     Marina|   26|
|        19|     Danilo|   36|
|        20|     Marina|   73|
+----------+-----------+-----+
only showing top 20 rows



In [ ]:
### 1. Conversão de Temperatura
### Uma UDF para converter temperaturas de Celsius para Fahrenheit e vice-versa pode ser 
### bastante útil em análises envolvendo dados climáticos ou ambientais.

def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32

def fahrenheit_to_celsius(fahrenheit):
    return (fahrenheit - 32) * 5/9

spark.udf.register("celsius_to_fahrenheit", celsius_to_fahrenheit, DoubleType())
spark.udf.register("fahrenheit_to_celsius", fahrenheit_to_celsius, DoubleType())

### 2. Tratamento de Strings
### UDFs para tratamento de strings, como a remoção de espaços em branco, 
### conversão para maiúsculas ou minúsculas, podem ser úteis em várias situações, especialmente na limpeza de dados.

def remove_whitespace(text):
    return text.strip()

def to_uppercase(text):
    return text.upper()

def to_lowercase(text):
    return text.lower()

spark.udf.register("remove_whitespace", remove_whitespace, StringType())
spark.udf.register("to_uppercase", to_uppercase, StringType())
spark.udf.register("to_lowercase", to_lowercase, StringType())

### 3. Cálculo de Distância
### Para conjuntos de dados geográficos, uma UDF para calcular a distância entre dois pontos 
### (latitude e longitude) pode ser muito útil.

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    # Convertendo coordenadas de graus para radianos
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Fórmula de Haversine
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Raio da Terra em quilômetros
    return c * r

spark.udf.register("haversine", haversine, DoubleType())

### 4. Tratamento de Datas
### UDFs para manipulação de datas, como calcular a diferença entre datas ou adicionar/subtrair 
### dias a uma data, são extremamente úteis em muitos contextos.

def days_between(date1, date2):
    return (date2 - date1).days

def add_days(date, days):
    return date + timedelta(days=days)

spark.udf.register("days_between", days_between, IntegerType())
spark.udf.register("add_days", add_days, DateType())